# Taxonomy analysis pipeline

This is the set of steps used to call taxonomy assignments for metagenomic reads from prostethic join infection samples. These steps were executed for each sample used in the study.


## Dependencies

The processing pipeline uses the following software, along with their respective dependencies:

* `samtools` v1.3, http://www.htslib.org/
* `biobloomtools` v2.0.12, https://github.com/bcgsc/biobloom
* `seqtk` latest, https://github.com/lh3/seqtk
* `Trimmomatic` v0.35, http://www.usadellab.org/cms/?page=trimmomatic

Specifically, for taxonomy assignment, we use

* `LMAT` v1.2.6, https://computation.llnl.gov/projects/livermore-metagenomics-analysis-toolkit
* `MetaPhlAn2` latest, https://bitbucket.org/biobakery/metaphlan2

## Pipeline
### Read clean-up

In [ ]:
#SAMPLE is the sample name
SAMPLE="sample_name"
#NSLOTS is the number of CPUs to be used

Extract reads in fastq format from BAM files (our source files are BAM-formatted)

In [ ]:
samtools bam2fq -1 ${SAMPLE}_R1.fastq -2 ${SAMPLE}_R2.fastq /path/to/${SAMPLE}.bam

Remove sequencing adapters and basic removal of low-quality bases from each read.

In [ ]:
#TRIMMOMATIC_JAR is the location of the trimmomatic.jar files 
TRIMMOMATIC_JAR=/path/to/trimmomatic-0.35.jar
#ADAPTERS is a trimmomatic-compatible file with sequencing adapters. Provided with Trimmomatic
ADAPTERS=/path/to/trimmomatic-0.35/adapters.fasta

java -jar $TRIMMOMATIC_JAR PE -threads $NSLOTS -phred33 \
${SAMPLE}_R1.fastq ${SAMPLE}_R2.fastq \
${SAMPLE}_R1.noadapter.fastq ${SAMPLE}_U1.noadapter.fastq \
${SAMPLE}_R2.noadapter.fastq ${SAMPLE}_U2.noadapter.fastq \
ILLUMINACLIP:$ADAPTERS:2:30:10 LEADING:3 TRAILING:3 MAXINFO:220:0.1 MINLEN:70

#keep orphaned/unpaired reads together
cat ${SAMPLE}_U[12].noadapter.fastq > ${SAMPLE}_U.noadapter.fastq

### Remove human reads, stage 1 of 2

`human.bf` and `phix.bf` are databases for `biobloomtools` created using the genomes of _Homo sapiens_ (hg19) and the _Enterobacteria phage ΦX174_, to remove human reads and leftover Illumina phiX control reads.

In [ ]:
HUMAN=/path/to/human.bf
PHIX=/path/to/phix.bf
#create working directory for biobloomtools output
mkdir bloom

#run biobloomtools
biobloomcategorizer -t $NSLOTS -p bloom/$SAMPLE -f "$HUMAN $PHIX" --fq -e ${SAMPLE}_R1.noadapter.fastq ${SAMPLE}_R2.noadapter.fastq
biobloomcategorizer -t $NSLOTS -p bloom/$SAMPLE -f "$HUMAN $PHIX" --fq ${SAMPLE}_U.noadapter.fastq


### `LMAT` taxonomy analysis

`LMATDB` and `LMATGENEDB` are databases for `LMAT` taxonomy call and gene call steps, respectively. Download them using the `LMAT`-provided scripts.

`merge_fastq_reads_with_N_separator.pl`, `run_rl.sh` and `run_gl.sh` are provided with `LMAT`.

In [ ]:
#set database locations
LMATDB=/path/to/kML+Human.v4-14.20.g10.db
LMATGENEDB=/path/to/lmat.genes.7-14.db

#join paired reads in preparation for LMAT
#mask bases with PHRED score lower than 10 using an N
merge_fastq_reads_with_N_separator.pl bloom/${SAMPLE}_noMatch_1.fq bloom/${SAMPLE}_noMatch_2.fq /dev/stdout | 
seqtk seq -A -q 10 -n N > ${SAMPLE}_merged.fasta
seqtk seq -A -q 10 -n N bloom/${SAMPLE}_noMatch.fq >> ${SAMPLE}_merged.fasta

#${SAMPLE}_merged.fasta will be the input to LMAT
#LMATOUT is the output directory for LMAT
LMATOUT=lmat_output
mkdir $LMATOUT
run_rl2.sh --threads=$NSLOTS --query_file=${SAMPLE}_merged.fasta --db_file=$LMATDB --odir=$LMATOUT

cd $LMATOUT

#prepare for LMAT-genes

FSUMM=$(ls *.fastsummary)
ls *out > ${SAMPLE}_rl_out.txt
run_gl.sh --threads=$NSLOTS --db_file=$LMATGENEDB --odir=. --ilst=${SAMPLE}_rl_out.txt --filesum=$FSUMM
rm -v $LMATGENEDB
GSUMM=$(ls *.genesummary)

cd ..


### Remove human reads, stage 2 of 2
Now that LMAT has executed fully, remove reads assigned to _Homo sapiens_.

`remove_ids_from_fastq.py` is a custom script provided in this repository, which removes reads from a fastq file given a list of read IDs.

In [ ]:
awk '{if ($(NF-2) == 9606 && $(NF) == "DirectMatch") print $1}' $LMATOUT/*.out > human.accnos

remove_ids_from_fastq.py human.accnos -i bloom/${SAMPLE}_noMatch_1.fq -o ${SAMPLE}_R1.clean.fastq
remove_ids_from_fastq.py human.accnos -i bloom/${SAMPLE}_noMatch_2.fq -o ${SAMPLE}_R2.clean.fastq
remove_ids_from_fastq.py human.accnos -i bloom/${SAMPLE}_noMatch.fq -o ${SAMPLE}_U.clean.fastq

### `MetaPhlAn2` taxonomy analysis
`v20_m200` refers to the name of the default database for `MetaPhlAn2`

In [ ]:
metaphlan2.py --nproc $NSLOTS  -x v20_m200 \
--sample_id $SAMPLE -o $SAMPLE.metaphlan.txt --bowtie2out $SAMPLE.bowtie2.out \
--input_type fastq ${SAMPLE}_R1.clean.fastq,${SAMPLE}_R2.clean.fastq,${SAMPLE}_U.clean.fastq

### Outputs
The output files are:
* `$SAMPLE.metaphlan.txt` from `MetaPhlAn2` (and `$SAMPLE.bowtie2.out` for further exploration)
* The `*.fastsummary` files inside the `$LMATOUT` directory. The `*.out` files can be used to verify hits, but they can take up a lot of space.

### Copyright
Copyright © 2018, Mayo Foundation for Medical Education and Research. All rights reserved.